In [94]:
import tensorflow as tf 
from tensorflow.keras.applications import MobileNetV2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt 
import numpy as np 
from scipy.spatial.distance import cosine

In [95]:
img_shape = (160, 160, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

neural_network = tf.keras.Sequential([
  base_model,
  global_average_layer,
])
def prepare_image(img_path, height = 160, width = 160, where = 'local'):
    
    img = tf.io.read_file(img_path)
    
    img = tf.image.decode_image(img)
    
    img = tf.cast(img, tf.float32)
    
    img = (img/127.5)-1
    
    img = tf.image.resize(img, (height, width))
    
    
    if img.shape!=(160, 160, 3):
        img = tf.concat([img, img, img], axis = 2)

    return img

def extract_features(image, neural_network):
    image_np = image.numpy()
    image_np = np.expand_dims(image_np, axis = 0)
    deep_features = neural_network.predict(image_np)[0]
    return deep_features

In [96]:
import pandas as pd
recommender_df = pd.read_csv('iter1_recommender.csv')
pics = pd.read_csv('pics.csv')
recommender_df.head()

,Unnamed: 0,image_local_name,hashtags,deep_features,id,features
0,471,cc6687dc-e2e5-417b-9372-faa5527ed25d.jpg,"['#monsieurbebelife', '#Ivorianbaby', '#monsie...","[0.2264766, 0.042613965, 0.7752272, 0.3973327,...",471,"[-0.15784144401550293, 0.5163793563842773, -0...."
1,463,bf96dc78-d4f8-4f35-a2ec-9605803a37c7.jpg,"['#cats', '#futuremaman', '#birman', '#maman',...","[0.0, 1.7325919, 0.12030285, 0.32206786, 0.316...",463,"[-0.057306136935949326, -0.2672276794910431, -..."
2,833,3b019eed-2e12-4e9a-a49c-ce131257e71e.jpg,"['#pet', '#godoxlighting', '#portraitphotograp...","[0.0, 0.0, 0.029408284, 0.0, 0.0, 0.0, 0.0, 0....",833,"[-0.3648046553134918, -0.5840925574302673, 0.5..."
3,148,08c86967-0dec-45d0-947e-8919b342714b.jpg,"['#nature', '#CzechGirl', '#BeautyOfNature', '...","[0.0, 0.47515857, 0.054196358, 0.98798513, 0.4...",148,"[-0.11143600195646286, -0.14699919521808624, 0..."
4,540,415c35ed-6177-4be1-b6d1-59e689acb1e9.jpg,"['#td', '#gqpatrol', '#gq', '#kettle', '#patrol']","[0.0, 0.0, 1.2779106, 0.0, 0.0, 2.1347184, 1.9...",540,"[0.0040685334242880344, -0.0004095354815945029..."


In [97]:
def find_neighbor_vectors(image_path, k=5, recommender_df=recommender_df):
    """Find image features (user vectors) for similar images."""
    prep_image = prepare_image(image_path, where='local')
    pics = extract_features(prep_image, neural_network)
    rdf = recommender_df.copy()
    rdf['dist'] = rdf['deep_features'].apply(lambda x: cosine(x, pics))
    rdf = rdf.sort_values(by='dist')
    return rdf.head(k)

In [98]:
def generate_hashtags(image_path):
    fnv = find_neighbor_vectors(image_path, k=5, recommender_df=recommender_df)
    # Find the average of the 5 user features found based on cosine similarity.
    features = []
    for item in fnv.features.values:
        features.append(item)

    avg_features = np.mean(np.asarray(features), axis=0)
    
    # Add new column to the hashtag features which will be the dot product with the average image(user) features
    hashtag_features['dot_product'] = hashtag_features['features'].apply(lambda x: np.asarray(x).dot(avg_features))

    # Find the 10 hashtags with the highest feature dot products
    final_recs = hashtag_features.sort_values(by='dot_product', ascending=False).head(10)
    # Look up hashtags by their numeric IDs
    output = []
    for hashtag_id in final_recs.id.values:
        output.append(hashtags_df.iloc[hashtag_id]['hashtag'])
    return output

In [99]:
def show_results(test_image):
    img = mpimg.imread(f'test/{test_image}.jpeg')
    plt.figure(figsize=(9, 9))
    plt.title(f'File Name: {test_image.upper()}', fontsize=32)        
    plt.imshow(img)
    
    recommended_hashtags = generate_hashtags(f'test/{test_image}.jpeg')
    print(', '.join(recommended_hashtags))

In [101]:
from ast import literal_eval
import numpy
for i in range(len(recommender_df['deep_features'])):
    print(i)
    recommender_df['deep_features'][i] = numpy.array(literal_eval(recommender_df['deep_features'][i]))

    

0


ValueError: malformed node or string: array([0.2264766 , 0.04261396, 0.7752272 , ..., 0.52041703, 0.        ,
       0.07019943])

In [93]:

for i in range(len(pics['deep_features'])):
    pics['deep_features'][i] = numpy.array(literal_eval(pics['deep_features'][i]))

SyntaxError: invalid syntax (<unknown>, line 1)

In [90]:
type(numpy.array(literal_eval(recommender_df['deep_features'][0])))

numpy.ndarray

In [47]:
np.array(recommender_df['deep_features'][0])

array('[0.2264766  0.04261396 0.7752272  ... 0.52041703 0.         0.07019943]',
      dtype='<U71')

In [76]:
list = recommender_df['deep_features'][0]

In [77]:
list

'[0.2264766  0.04261396 0.7752272  ... 0.52041703 0.         0.07019943]'